In [5]:
# !nvidia-smi

## 1. Import Libraries

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os

from PIL import Image
from typing import List, Tuple
import random
from pathlib import Path
import numpy as np

### Check GPU Availability

In [7]:
# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")
print(f"PyTorch version: {torch.__version__}")

# Display GPU-info if available
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"GPU: {torch.cuda.get_device_name(0)} is Available.")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

Using Device: cuda
PyTorch version: 2.8.0+cu126
GPU: NVIDIA GeForce GTX 1650
GPU Memory: 4.3 GB
GPU: NVIDIA GeForce GTX 1650 is Available.
Number of GPUs: 1


### 1. Custom Dataset Class for OOD Detection


In [8]:
class CarDataset(Dataset):
    """Dataset for loading car images and OOD samples"""
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.samples = []

        # car classes
        car_classes = [
            'Maruti_Suzuki_Baleno',
            'Maruti_Suzuki_Brezza',
            'Maruti_Suzuki_Swift',
            'Maruti_Suzuki_WagonR'
        ]

        # Add car sample
        for idx, class_name in enumerate(car_classes):
            class_path = os.path.join(root_dir, class_name)

            if os.path.exists(class_path):
                for img_name in os.listdir(class_path):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append({
                            'path': os.path.join(class_path, img_name),
                            'car_class': idx,  # 0,1,2,3
                            'is_car': 1        # Binary: this is a car
                        })
                        
        # Add OOD samples (Not_Car folder)
        ood_path = os.path.join(root_dir, "Not_car")
        if os.path.exists(ood_path):
            for img_name in os.listdir(ood_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append({
                        'path': os.path.join(ood_path, img_name),
                        'car_class': -1,  # No car class
                        'is_car': 0       # Binary: This is Not car
                    })

        print(f"Load {len(self.samples)} samples from {root_dir}")

        # count_distribution 
        car_count = 0
        ood_count = 0

        for sample in self.samples:
            if sample["is_car"] == 1:
                car_count += 1
            else:
                ood_count += 1

        print(f"Samples in our self.samples {self.samples[:5]}")

        print(f"Car Images: {car_count}, OOD Images: {ood_count}")

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self,idx):
        sample = self.samples[idx]
        image = Image.open(sample['path']).convert('RGB')

        if self.transform:
            image = self.transform(image)
        
        return image, sample['car_class'], sample['is_car']

## 2. Multi-Head Model Architecture

### Inspect Original EfficientNet-B0 Structure


In [15]:
from torchinfo import summary

In [18]:
# Create original EfficientNet-B0 for inspection
original_efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

print("-" * 50)
print("ORIGINAL EFFICIENTNET-B0 STRUCTURE") 
print("-" * 50)

# Display Detailed Summary
summary(model=original_efficientnet,
        input_size=(32,3,224,224), # batch_size, channels, heights, width
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width=47,
        row_settings=["var_names"])

--------------------------------------------------
ORIGINAL EFFICIENTNET-B0 STRUCTURE
--------------------------------------------------


Layer (type (var_name))                                      Input Shape                                     Output Shape                                    Param #                                         Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]                               [32, 1000]                                      --                                              True
├─Sequential (features)                                      [32, 3, 224, 224]                               [32, 1280, 7, 7]                                --                                              True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]                               [32, 32, 112, 112]                              --                                              True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]                               [32, 32, 112, 112]                            

### Multi-Head Model Architecture

In [ ]:
class CarClassifierWithOOD(nn.Module):
    """Multi-head neural network for car classification and OOD detection"""

    def __init__(self, num_car_classes=4, pretrained=True):
        super(CarClassifierWithOOD, self).__init__()

        # Load EfficientNet-B0 backbone
        if pretrained:
            weights = models.EfficientNet_B0_Weights.DEFAULT
        else:
            weights = None

        self.backbone = models.efficientnet_b0(weights=weights)

        # Remove the original classfier
        self.backbone.classifier = nn.Identity()

        # freeze base layers
        for param in self.backbone.features.parameters():
            param.requires_grad = True
        
        




SyntaxError: expected ':' (4024436916.py, line 19)